In [106]:
%cd C:\Users\yukir\Documents\GitHub\Sign_Recognition\py

from basic_preprocessing import *
from deeplearning_check import *
from machine_learning import *

gpu_check()
set_global_determinism(seed=SEED)

import os
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
import matplotlib.pylab as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import glob
import math
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
from IPython.display import Image

#cv
import cv2
import math
from PIL import Image
import math
from scipy import ndimage
import argparse
import imutils
import xml.etree.ElementTree as ET


#시각화
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
get_ipython().run_line_magic('matplotlib', 'inline')

import matplotlib.image as mpimg
from matplotlib import font_manager, rc
rc('font',family="consolas")
plt.rcParams["font.family"]="consolas" #plt 한글꺠짐
plt.rcParams["font.family"]="Arial" #외국어꺠짐
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 출력 설정
plt.rc('figure', figsize=(10,8))

sns.set(font="consolas", 
        rc={"axes.unicode_minus":False},style='darkgrid')


C:\Users\yukir\Documents\GitHub\Sign_Recognition\py
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14192974683001719439
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6635483358035173645
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9883535296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9707257092248114140
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 993142483275979442
physical_device_desc: "device: XLA_GPU device"
]
tf 2.2.0
keras 2.3.0-tf
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라


In [107]:
import random 
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize

%cd C:\Users\yukir\Documents\Monicas_workspace\Knee

C:\Users\yukir\Documents\Monicas_workspace\Knee


In [108]:
def show(img):
    #사이즈
    plt.figure(figsize = (6,6))
    #xticks/yticks - 눈금표
    plt.xticks([])
    plt.yticks([])
    #코랩에서 안돌아감 주의
    plt.imshow(img, cmap= 'gray')
    plt.show()

def show_img_compar(img_1, img_2 ):
    f, ax = plt.subplots(1, 2, figsize=(6,6))
    ax[0].imshow(img_1)
    ax[1].imshow(img_2)
    ax[0].axis('off') #hide the axis
    ax[1].axis('off')
    f.tight_layout()
    plt.show()

#### MODEL_LOAD

In [109]:
from tensorflow.keras.models import load_model
model = load_model('C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\unet\\[0110]knee_annotation.h5')

#### IMG_LOADER

In [132]:
# img_loader
#Test Image
IMG_WIDTH = 224
IMG_HEIGHT = 224
IMG_CHANNELS = 1

IMG_PATH = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\knee_dataset_org\\4\\'
img_list = []
file_name =[]
for i in os.listdir(IMG_PATH):
    adr = IMG_PATH+i
    # print(adr)
    img_list.append(IMG_PATH+i)
    file_name.append(i)

print(len(img_list))


295


In [133]:
# img_loader
#Test Image

X_test = np.zeros((len(img_list),224,224), dtype= np.uint8)

sizes_test = []
for n, id_ in tqdm(enumerate(img_list)):
    img_ = imread(id_)
    # show(img)
    hist, bins = np.histogram(img_.flatten(), 256,[0,256])

    cdf = hist.cumsum()

    # cdf의 값이 0인 경우는 mask처리를 하여 계산에서 제외
    # mask처리가 되면 Numpy 계산에서 제외가 됨
    # 아래는 cdf array에서 값이 0인 부분을 mask처리함
    cdf_m = np.ma.masked_equal(cdf,0)

    #History Equalization 공식
    cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())

    # Mask처리를 했던 부분을 다시 0으로 변환
    cdf = np.ma.filled(cdf_m,0).astype('uint8')

    img = cdf[img_]
    # print(img.shape)
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT,IMG_WIDTH), mode = 'constant', preserve_range = True)
    X_test[n] = img

print('Done')

295it [00:02, 144.34it/s]

Done


In [134]:
preds_test = model.predict(X_test, verbose =1)
preds_test_t = (preds_test>0.5).astype(np.uint8)

10/10 [==============================] - 0s 35ms/step


In [135]:
for i in tqdm (range (len(preds_test_t))):
    img_1 = imread(img_list[i])
    img_2 = X_test[i]
    img_3 = np.squeeze(preds_test_t[i])

    img_name = file_name[i].split('.jpg')[0]

    #저장
    save_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\mask_dataset\\4\\'
    img = cv2.convertScaleAbs(img_3, alpha=(255.0)) # 이거없으면 all black 으로 저장됨 
    cv2.imwrite(save_path+img_name+'_mask.jpg', img)

    # print(len(preds_test_t)-i, '--------', file_name[i])
    # show(preds_test_t[i])
    # f, ax = plt.subplots(1, 3, figsize=(8,8))
    # ax[0].imshow(img_1,cmap = 'gray')
    # ax[1].imshow(img_2,cmap = 'gray')
    # ax[2].imshow(img_3,cmap = 'gray')
    # ax[0].axis('off') #hide the axis
    # ax[1].axis('off')
    # ax[2].axis('off')
    # f.tight_layout()
    # plt.show()

100%|██████████| 295/295 [00:01<00:00, 240.81it/s]
